### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq 
!pip install -U sagemaker

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!rm -rf ./LLM_qwen_int4_model

In [12]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_qwen_int4_model")


In [4]:
local_model_path.mkdir(exist_ok=True)
model_name = "Qwen/Qwen-7B-Chat-Int4"
commit_hash = "955bbfa8c8f0c592b7713f1186c2c9ab477ef862"

In [5]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862'

### 2. 把模型拷贝到S3为后续部署做准备

In [10]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [13]:
s3_model_prefix = "LLM-RAG/workshop/LLM_qwen_int4_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code
model_snapshot_path: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862


In [8]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_model/.gitattributes
upload: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_model/README.md
upload: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862/LICENSE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_model/LICENSE
upload: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862/NOTICE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_model/NOTICE
upload: LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/955bbfa8c8f0c592b7713f1186c2c9ab477ef862/assets/performance.png to

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [22]:
#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
# )


inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [12]:
!mkdir -p LLM_qwen_int4_stream_deploy_code

In [114]:
%%writefile LLM_qwen_int4_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoTokenizer
from transformers.generation import GenerationConfig
from auto_gptq import AutoGPTQForCausalLM


STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    model = AutoGPTQForCausalLM.from_quantized(model_location, device_map="auto", trust_remote_code=True, use_safetensors=True).eval()
    config = GenerationConfig.from_pretrained(model_location, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
    return model, tokenizer, config


model = None
tokenizer = None
generator = None
config = None

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer, config
    size = 0
    response = ""
    config.max_new_tokens = max_length
    config.top_p = top_p
    
    ##传入temperature会报错
    ##model.generation_config.temperature = temperature 
    res_generator = model.chat_stream(tokenizer, prompt, history=history,generation_config=config)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}


def handle(inputs: Input):
    global model, tokenizer,config
    if not model:
        model, tokenizer,config = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data.get("history",[])
    stream = data.get('stream')
    print(f'input prompt:{input_sentences}')   
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        config.max_new_tokens = params.get('max_length',1024)
        config.top_p = params.get('top_p',1)
        response, history = model.chat(tokenizer, input_sentences, history=history,generation_config=config)
        result = {"outputs": response, "history" : history}
        outputs.add_as_json(result)
        
    return outputs

Overwriting LLM_qwen_int4_stream_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [115]:
%%writefile LLM_qwen_int4_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_model/

Overwriting LLM_qwen_int4_stream_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [116]:
%%writefile LLM_qwen_int4_stream_deploy_code/requirements.txt
transformers==4.31.0
accelerate
tiktoken
einops
transformers_stream_generator==0.0.4
scipy
gekko
auto-gptq

Overwriting LLM_qwen_int4_stream_deploy_code/requirements.txt


In [117]:
# !pip install auto-gptq

In [118]:
!rm model.tar.gz
!cd LLM_qwen_int4_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_qwen_int4_stream_deploy_code

LLM_qwen_int4_stream_deploy_code/
LLM_qwen_int4_stream_deploy_code/model.py
LLM_qwen_int4_stream_deploy_code/requirements.txt
LLM_qwen_int4_stream_deploy_code/serving.properties


In [119]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [120]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"qwen-stream-int4") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

qwen-stream-int4-2023-08-31-02-25-35-582
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/qwen-stream-int4-2023-08-31-02-25-35-582


In [121]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/qwen-stream-int4-2023-08-31-02-25-35-582-config',
 'ResponseMetadata': {'RequestId': '03bddef3-a6dc-4271-9625-67fba840982a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '03bddef3-a6dc-4271-9625-67fba840982a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '128',
   'date': 'Thu, 31 Aug 2023 02:25:38 GMT'},
  'RetryAttempts': 0}}

In [122]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-stream-int4-2023-08-31-02-25-35-582-endpoint


#### 持续检测模型部署进度

In [123]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-stream-int4-2023-08-31-02-25-35-582-endpoint
Status: InService


### 5. 模型测试

In [5]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.1,
  "top_p":0.8
}

CPU times: user 288 ms, sys: 45.8 ms, total: 334 ms
Wall time: 419 ms


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [6]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream

In [9]:
# prompts1 = """你"""
import time

start = time.time()
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
                "stream":True
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            print(line)
            continue
print (f"\ntime:{time.time()-start} s")

故事发生在遥远的未来，地球已被毁灭，人类为了生存，不得不开始寻找新的家园。在宇宙中，他们发现了无数的星球，但是每个星球都有不同的危险。于是，人类开始建立联盟，共同抵抗宇宙中的敌人。

联盟中最强大的一支军队，由人类最聪明的人组成，他们被称为“智慧军”。智慧军的任务是保护联盟的星球，同时也探索宇宙中的秘密。他们乘坐着最先进的飞船，穿梭在宇宙中，寻找敌人的踪迹。

有一天，智慧军接到了一个紧急任务。他们接到了来自遥远星球的求救信号。这个星球被一种强大的外星种族占领，他们威胁要毁灭整个星球。智慧军立即出发，前往这个星球。

当他们到达星球时，他们发现整个星球已经被外星种族的基地所包围。智慧军需要想出一个聪明的计划来击败外星种族。

经过一番讨论，智慧军制定出了一个大胆的计划。他们利用星球上的自然元素，制造出了无数的机器人和无人机，这些机器人和无人机可以攻击外星种族的基地。然后，智慧军派遣了一些精英士兵潜入外星种族的基地，破坏了他们的电脑系统。

外星种族的基地受到了攻击，他们非常惊慌失措。但是他们很快恢复了秩序，并开始反击。智慧军的机器人和无人机与外星种族的机器人和战机展开了激战。经过一番苦战，智慧军最终获得了胜利。

智慧军拯救了这个星球，并获得了外星种族的尊重。他们与外星种族建立了友好的关系，并开始共同探索宇宙中的秘密。
time:11.242651462554932 s


## None stream

In [8]:

endpoint_name= 'qwen-stream-int4-2023-08-31-02-25-35-582-endpoint'
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
start = time.time()
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

resp = response_model['Body'].read()
print (f"\ntime:{time.time()-start} s")
# print(resp.decode('utf8'))
print(json.loads(resp)['outputs'])



time:12.590379476547241 s
在遥远的宇宙中，有两个强大的文明，分别是赛拉克斯文明和阿瑞斯文明。这两个文明之间的矛盾已经持续了很长时间，双方之间的战争也从未停歇。

赛拉克斯文明是一个科技高度发达的文明，他们拥有一种叫做“能源炮”的武器，可以发射出强大的能量束，瞬间摧毁敌人的防御系统。而阿瑞斯文明则更注重实战经验，他们的战士们有着强大的体能和战斗技巧，可以与赛拉克斯文明的高科技武器相抗衡。

在一场决定宇宙命运的战争中，赛拉克斯文明的能源炮被阿瑞斯文明的战士们成功摧毁，让赛拉克斯文明的军队陷入了困境。但赛拉克斯文明并没有放弃，他们开始研发一种新型的能源武器，这种武器可以比能源炮更加先进和强大。

经过几个月的艰苦研发，赛拉克斯文明终于成功地制造出了新型的能源武器。这种武器不仅可以发射出更强大的能量束，还可以进行智能化攻击，可以根据敌人的情况进行变化和调整。

在另一场决定宇宙命运的战争中，赛拉克斯文明的新型能源武器终于派上了用场。在激烈的战斗中，赛拉克斯文明的军队成功地使用新型能源武器摧毁了阿瑞斯文明的防御系统，并将阿瑞斯文明的军队彻底击败。

随着赛拉克斯文明的胜利，宇宙的和平也终于来临。赛拉克斯文明的科学家们也开始致力于探索宇宙的奥秘，希望可以找到更多的先进技术和知识，让宇宙更加美好。而阿瑞斯文明的战士们则开始重新学习战斗技巧，希望可以在未来的战争中重新崛起。

在宇宙的漫长岁月中，战争和和平一直在循环。但只有那些不断探索和学习的文明，才能在宇宙中生存和繁衍下去。


#### 清除模型Endpoint和config

In [111]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [112]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [113]:
!aws sagemaker delete-model --model-name {model_name}